In [ ]:
import tensorflow as tf 
import tensorflow_datasets as tfds # to load training data


In [ ]:
!pip install tensorflow_datasets

In [ ]:
import tensorflow_datasets as tfds
DATA_DIR="./tensorflow-datasets"


In [ ]:

tf.random.set_seed(1024)



#hyperparameter 1 batch_size = 8, batch_size = 32
#hyperparameter 2 optimizer=adam or sgd with default learning rate
early_stopping_callback = tf.keras.callbacks.EarlyStopping(patience=3, restore_best_weights=True)
csv_logger = tf.keras.callbacks.CSVLogger(filename="experiment_log.csv")
model.compile(optimizer=tf.keras.optimizers.Adam(),loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),metrics=['accuracy'])


model.summary()


In [ ]:
batch_size = 32

train_ds= tfds.load('fashion_mnist', split="train[:80%]",data_dir=DATA_DIR, shuffle_files=False).map(lambda x: (tf.cast(x['image'],tf.float32)/255.0, x['label'])).batch(batch_size)
val_ds= tfds.load('fashion_mnist', split="train[-20%:]",data_dir=DATA_DIR, shuffle_files=False).map(lambda x: (tf.cast(x['image'],tf.float32)/255.0, x['label'])).batch(batch_size)

# val_images,val_labels = tfds.load('fashion_mnist',split="train[-20%:]",shuffle_files=False,as_supervised=True).batch(batch_size)






In [ ]:
model.fit(train_ds,validation_data=val_ds,epochs=100,callbacks=[early_stopping_callback,csv_logger])

In [ ]:
import utils 

import numpy as np 
from sklearn.metrics import accuracy_score,f1_score,confusion_matrix


def evaluate(model):
    test_ds = utils.get_test_set()
    predictions = model.predict(test_ds).argmax(axis=-1)
    gt = np.concatenate([y for _, y in test_ds], axis=0)
    acc_score = accuracy_score(gt,predictions)
    f1 = f1_score(gt,predictions,average="macro")
    cf = confusion_matrix(gt,predictions)
    return acc_score,f1,cf






In [ ]:
results = evaluate(model)

In [ ]:
import pickle 

pickle.dump(results,open("results","wb"))

In [ ]:
results_1 = pickle.load(open("results","rb"))

In [ ]:
pred

Working on model arch 1,[batch_8,adam,regularizer]


2024-09-29 23:16:44.714341: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :
2024-09-29 23:16:44.714375: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2024-09-29 23:16:44.714396: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (c3618.swan.hcc.unl.edu): /proc/driver/nvidia/version does not exist
2024-09-29 23:16:44.745915: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-29 23:16:45.524495: W tensorflow/core/platform/cloud/g

In [ ]:
%run main.py

Working on model arch 1,[batch_8,adam,regularizer]


2024-09-30 08:34:30.652389: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :
2024-09-30 08:34:30.652425: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2024-09-30 08:34:30.652444: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (c3618.swan.hcc.unl.edu): /proc/driver/nvidia/version does not exist
2024-09-30 08:34:30.693964: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-30 08:34:31.795942: I tensorflow/compiler/mlir/mlir_gr

Epoch 1/100
6000/6000 [==============================] - 15s 2ms/step - loss: 1.7875 - accuracy: 0.6995 - val_loss: 1.2336 - val_accuracy: 0.7452
Epoch 2/100
6000/6000 [==============================] - 9s 2ms/step - loss: 1.2236 - accuracy: 0.7379 - val_loss: 1.1735 - val_accuracy: 0.7481
Epoch 3/100
6000/6000 [==============================] - 9s 2ms/step - loss: 1.1798 - accuracy: 0.7434 - val_loss: 1.1447 - val_accuracy: 0.7479
Epoch 4/100
6000/6000 [==============================] - 9s 2ms/step - loss: 1.1549 - accuracy: 0.7459 - val_loss: 1.1332 - val_accuracy: 0.7479
Epoch 5/100
5974/6000 [============================>.] - ETA: 0s - loss: 1.1392 - accuracy: 0.7480